# Assignment 3 - Transliteration

### Setup [do not change]

In [ ]:
!git clone https://github.com/butoialexandra/eth-nlp-f22-hw3.git

import os
os.chdir('eth-nlp-f22-hw3')

!pip install -e .

Cloning into 'eth-nlp-f22-hw3'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 36 (delta 9), reused 24 (delta 4), pack-reused 0
Receiving objects: 100% (36/36), 11.93 KiB | 3.98 MiB/s, done.
Resolving deltas: 100% (9/9), done.
Obtaining file:///content/eth-nlp-f22-hw3/eth-nlp-f22-hw3
  Preparing metadata (setup.py) ... done
  Attempting uninstall: rayuela
    Found existing installation: rayuela 0.1
    Uninstalling rayuela-0.1:
      Successfully uninstalled rayuela-0.1
  Running setup.py develop for rayuela


In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1wlHbFChZFcBgEU8knAeaUIqE0W4EaJEC' -O 'test_cases.pkl'

--2024-01-14 22:52:22--  https://docs.google.com/uc?export=download&id=1wlHbFChZFcBgEU8knAeaUIqE0W4EaJEC
Resolving docs.google.com (docs.google.com)... 142.250.31.102, 142.250.31.113, 142.250.31.139, ...
Connecting to docs.google.com (docs.google.com)|142.250.31.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1wlHbFChZFcBgEU8knAeaUIqE0W4EaJEC&export=download [following]
--2024-01-14 22:52:22--  https://drive.usercontent.google.com/download?id=1wlHbFChZFcBgEU8knAeaUIqE0W4EaJEC&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.167.132, 2607:f8b0:4004:c1d::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.167.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43862 (43K) [application/octet-stream]
Saving to: ‘test_cases.pkl’

test_cases.pkl      100%[===================>]  42.83K  --.-KB/s    in

In [ ]:
from rayuela.base.semiring import Semiring, Real, Tropical
from rayuela.base.symbol import Sym, ε, ε_1, ε_2
from rayuela.fsa.fsa_classes import get_epsilon_filter

from rayuela.fsa.fst import FST
from rayuela.fsa.state import State, PairState

import dill
import numpy as np
from math import exp, log

from itertools import product

from frozendict import frozendict

### Subquestion a) - Log Semiring

In [ ]:
class Log(Semiring):
    def __init__(self, score):
        self.score = score

    def star(self):
        # TODO: implement Kleene star
        if self.score >= 0:
            return Log(np.inf)
        else:
            return Log(- np.log(1 - np.exp(self.score)))

    def __float__(self):
        return float(self.score)

    def __add__(self, other):
        # TODO: implement log addition
        # For numerical stability you should use the using the log-sum-exp trick
        # https://en.wikipedia.org/wiki/LogSumExp
        return Log(np.logaddexp(self.score, other.score))

    def __mul__(self, other):
        # TODO: implement log multiplication
        return Log(self.score + other.score)

    def __repr__(self):
        return f"{round(self.score, 15)}"

    def __eq__(self, other):
        return np.allclose(float(self.score), float(other.score), atol=1e-3)

    def __hash__(self):
        return hash(self.score)

# TODO: implement the log semiring one and zero
Log.zero = Log(- np.inf)
Log.one = Log(0)


Log.idempotent = False
Log.cancellative = True

### Before moving to the next subquestion, we give some examples of WFSTs to illustrate how to use the library `rayuela`.

In [ ]:
# A simple language model encoded by a WFST

sem = Real # the Real semiring
fst = FST(sem) # a WFST in the Real semiring

# add transitions
fst.add_arc(State(0), Sym('formal'), Sym('formal'), State(1), Real(0.2))
fst.add_arc(State(0), Sym('natural'), Sym('natural'), State(1), Real(0.3))
fst.add_arc(State(0), Sym('learning'), Sym('learning'), State(2), Real(0.2))
fst.add_arc(State(0), Sym('data'), Sym('data'), State(3), Real(0.3))

fst.add_arc(State(1), Sym('language'), Sym('language'), State(2), Real(0.6))
fst.add_arc(State(1), Sym('languages'), Sym('languages'), State(4), Real(0.4))

fst.add_arc(State(2), Sym('is'), Sym('is'), State(5), Real(1.0))

fst.add_arc(State(5), Sym('fun'), Sym('fun'), State(6), Real(1.0))

fst.add_arc(State(3), Sym('is'), Sym('is'), State(5), Real(0.5))
fst.add_arc(State(3), Sym('are'), Sym('are'), State(5), Real(0.5))

fst.add_arc(State(4), Sym('are'), Sym('are'), State(5), Real(1.0))

# set initial weights
# if not set, it defaults to semiring zero
fst.set_I(State(0), Real(1.0))
# set final weights
fst.set_F(State(6), Real(1.0))

# visualize
fst

WFSA(7 states, <class 'rayuela.base.semiring.Real'>)

In [ ]:
# A simple WFSA for transliteration

sem = Real # semiring
fst = FST(sem) # initialize WFST in the chosen semiring
one, zero = sem.one, sem.zero # semiring one and zero

# add transitions
fst.add_arc(State(0), Sym('d'), Sym('data'), State(1), Real(0.5))
fst.add_arc(State(0), Sym('d'), Sym('dew'), State(5), Real(0.5))

fst.add_arc(State(1), Sym('ey'), Sym('ÃŽÂµ'), State(2), Real(0.5))
fst.add_arc(State(1), Sym('ae'), Sym('ÃŽÂµ'), State(2), Real(0.5))

fst.add_arc(State(2), Sym('t'), Sym('ÃŽÂµ'), State(3), Real(0.7))
fst.add_arc(State(2), Sym('dx'), Sym('ÃŽÂµ'), State(3), Real(0.3))

fst.add_arc(State(3), Sym('ax'), Sym('ÃŽÂµ'), State(4), one)

fst.add_arc(State(5), Sym('uw'), Sym('ÃŽÂµ'), State(6), one)

# initial and final weights
fst.set_I(State(0), one)
fst.set_F(State(4), one)
fst.set_F(State(6), one)

fst

WFSA(7 states, <class 'rayuela.base.semiring.Real'>)

### Subquestion b) - String Transducer

In [ ]:
weight = - np.exp(4)

In [ ]:
def string_fst(s, semiring=Log):
    """ Takes a string as input and a semiring
    and returns a transducer encoding of the
    string
    """
    fst = FST(semiring)
    one, zero = semiring.one, semiring.zero

    # TODO: add arcs, initial and final weights

    for idx, symbol in enumerate(s):
        fst.add_arc(State(idx), symbol, symbol, State(idx + 1), semiring(weight))

    fst.set_I(State(0), one)
    fst.set_F(State(len(s)), one)

    return fst

In [ ]:
# you can visualise the transducer
fst = string_fst("abc", semiring=Real)
fst

WFSA(4 states, <class 'rayuela.base.semiring.Real'>)

### Subquestion c) - Edit Distance Transducer

As we will not learn the weights on the arcs, you can set them to some pre-defined constants. You should choose the weights such that the pathsum that needs to be computed in e) does not diverge (this is caused by the weights of the self-loops in the transducer).

In [ ]:
def edit_distance_fst(source_alphabet, target_alphabet, semiring=Log):
    """ Takes as input two sets of characters (source and target
    alphabets) and a semiring and returns an edit-distance transducer
    """

    fst = FST(semiring)
    one, zero = semiring.one, semiring.zero

    # TODO: add arcs, initial and final weights
    for idx_1, symbol_1 in enumerate(target_alphabet):
        for symbol_2 in source_alphabet.union({ε}):
            for idx_2 in range(len(target_alphabet)):
                fst.add_arc(State(idx_2), symbol_2, symbol_1, State(idx_1), semiring(weight))

    for idx in range(len(target_alphabet)):
        fst.set_I(State(idx), one)
        fst.set_F(State(idx), one)

    return fst

In [ ]:
fst = edit_distance_fst({'a', 'b', 'c'}, {'d', 'e'}, semiring=Real)
fst

WFSA(2 states, <class 'rayuela.base.semiring.Real'>)

### Subquestion d) - Composition

We provide some composition test cases for you to inspect.

In [ ]:
with open('test_cases.pkl', 'rb') as in_file:
    examples = dill.load(in_file)

In [ ]:
# FST 1
examples[0]['fst1']['fst']

WFSA(2 states, <class 'rayuela.base.semiring.Real'>)

In [ ]:
# FST 2
examples[0]['fst2']['fst']

WFSA(2 states, <class 'rayuela.base.semiring.Real'>)

In [ ]:
# composition
examples[0]['composition']['fst']

WFSA(3 states, <class 'rayuela.base.semiring.Real'>)

In [ ]:
# semiring
examples[0]['semiring']

rayuela.base.semiring.Real

In [ ]:
def composition_algorithm(fst1, fst2):
    """Implementation of the Accessible Composition Algorithm as seen in the lecture"""

    # The machines must be in the same semiring
    assert fst1.R == fst2.R

    comp_fst = FST(R=fst1.R)

    stack = [(q1[0], q2[0]) for q1 in fst1.I for q2 in fst2.I]
    visited = set(stack)

    while len(stack) > 0:
        q1, q2 = stack.pop()

        for e1, e2 in product(fst1.arcs(q1), fst2.arcs(q2)):
            (a, b), q1_prime, w1 = e1
            (c, d), q2_prime, w2 = e2

            if b == c:
                comp_fst.set_arc(PairState(q1, q2), a, d, PairState(q1_prime, q2_prime), w1 * w2)

                if (q1_prime, q2_prime) not in visited:
                    visited.add((q1_prime, q2_prime))
                    stack.append((q1_prime, q2_prime))

    for q in comp_fst.Q:
        q1, q2 = q.idx
        comp_fst.λ[q] = fst1.λ[q1] * fst2.λ[q2]
        comp_fst.ρ[q] = fst1.ρ[q1] * fst2.ρ[q2]


    return comp_fst

In [ ]:
def compose(fst1, fst2):
    """ Takes as input two transducers and returns
    their composition
    """
    # The machines must be in the same semiring
    assert fst1.R == fst2.R

    comp_fst = FST(R=fst1.R)

    # TODO: implement composition

    # Utilizes the composition_algorithm implementation defined above to compute the composition of
    # the first FST with the FST generated by the get_epsilon_filter method, and the resulting FST
    # is further composed with the second FST to obtain the true and correct composition of FST1 and FST2

    fst1 = fst1.augment_epsilon_transitions(1)
    fst2 = fst2.augment_epsilon_transitions(2)

    epsilon_t = get_epsilon_filter(R=fst1.R, Sigma=fst1.Sigma)

    interm_fst = composition_algorithm(fst1, epsilon_t)
    comp_fst = composition_algorithm(interm_fst, fst2)

    return comp_fst

In [ ]:
examples[4]['composition']['fst']

WFSA(8 states, <class 'rayuela.base.semiring.Boolean'>)

In [ ]:
i = 4
fst = compose(examples[i]['fst1']['fst'], examples[i]['fst2']['fst'])
fst

WFSA(8 states, <class 'rayuela.base.semiring.Boolean'>)

### Subquestion e) - Lehmann's Algorithm

In [ ]:
class Pathsum:
    def __init__(self, fsa):

        # basic FSA stuff
        self.fsa = fsa
        self.R = fsa.R
        self.N = self.fsa.num_states

        # state dictionary
        self.I = {}
        for n, q in enumerate(self.fsa.Q):
            self.I[q] = n

        # lift into the semiring
        self.W, self.W_dict, self.alpha, self.beta = self.lift()


    def lift(self):
        """ creates the weight matrix, initial weight vector,
        final weight vector from the automaton """
        W = self.R.zeros((self.N, self.N))
        W_dict = {a: {} for a in self.fsa.Sigma}
        alpha = self.R.zeros((self.N,))
        beta = self.R.zeros((self.N,))
        for a in self.fsa.Sigma:
            for b in self.fsa.Delta:
                W_dict[a][b] = self.R.zeros((self.N, self.N))
        for p in self.fsa.Q:
            alpha[self.I[p]] += self.fsa.λ[p]
            beta[self.I[p]] += self.fsa.ρ[p]
            for (a, b), q, w in self.fsa.arcs(p):
                W[self.I[p], self.I[q]] += w
                W_dict[a][b][self.I[p], self.I[q]] = w
        return W, W_dict, alpha, beta

    def lehmann(self):
        """
        Lehmann's (1977) algorithm.
        """
        # TODO: implement
        U = self.W.copy()
        V = self.W.copy()

        for j in range(self.N):
            V = self.R.zeros((self.N, self.N))

            for i in range(self.N):
                for k in range(self.N):
                    V[i,k] = U[i,k] + U[i,j] * U[j,j].star() * U[j,k]

            U = V

        for i in range(self.N):
            V[i, i] += self.fsa.R.one

        return V

    def _iterate(self, K):
        P = self.R.diag(self.N)
        for n in range(K):
            P += self.W @ P
        return P

    def _fixpoint(self, K=200):
        if self.fsa.R.idempotent:
            return self._iterate(self.fsa.num_states)

        diag = self.R.zeros((self.fsa.num_states, self.fsa.num_states))
        for n in range(self.fsa.num_states):
            diag[n, n] = self.fsa.R.one
        P_old = diag

        for _ in range(K):
            P_new = diag + self.W @ P_old
            P_old = P_new

        return P_old

    def fixpoint(self):

        P = self._fixpoint()
        W = {}

        for p in self.fsa.Q:
            for q in self.fsa.Q:
                if p in self.I and q in self.I:
                    W[p, q] = P[self.I[p], self.I[q]]
                elif p == q:
                    W[p, q] = self.R.one
                else:
                    W[p, q] = self.R.zero

        return frozendict(W)

In [ ]:
pathsum = Pathsum(examples[0]['fst1']['fst'])
print(pathsum.fixpoint())
print(pathsum.lehmann())

frozendict.frozendict({(0, 0): 1.0, (0, 1): 0.25, (1, 0): 0.0, (1, 1): 1.146788990825688})
[[1.0 0.25]
 [0.0 1.146788990825688]]


### Subquestion f) - Normalizer

We provide some test cases for you to inspect.

In [ ]:
examples[0]['fst1']['Z'], examples[0]['fst2']['Z'], examples[0]['composition']['Z']

(0.00138425, 0.000997195294118, 7.07192371e-07)

In [ ]:
def normalizer(fst):
    """ Takes as input a finite-state transducer and
    returns its normalizer
    """

    pathsum = Pathsum(fst)
    # TODO: implement normalizer
    Z = fst.R.zero
    V = pathsum.lehmann()

    for n, q_n in enumerate(fst.Q):
        for m, q_m in enumerate(fst.Q):
            Z += fst.λ[q_n] * V[n,m] * fst.ρ[q_m]

    return Z

In [ ]:
print(normalizer(examples[0]['fst1']['fst']), normalizer(examples[0]['fst2']['fst']), normalizer(examples[0]['composition']['fst']))

0.00138425 0.000997195294118 7.07192371e-07


### Subquestion g) - Log-likelihood

In [ ]:
def log_likelihood(source_alphabet, target_alphabet, source_str, target_str):
    """ Takes as input the source and target alphabets (sets of characters)
    and a pair of source and target strings and returns the log-likelihood
    """

    # TODO: implement log-likelihood
    fst1 = string_fst(source_str, Log)
    fst2 = string_fst(target_str, Log)
    ed_fst = edit_distance_fst(source_alphabet, target_alphabet, Log)

    fst1_ed_fst = compose(fst1, ed_fst)
    fst1_ed_fst_fst2 = compose(fst1_ed_fst, fst2)

    return normalizer(fst1_ed_fst_fst2).score - normalizer(fst1_ed_fst).score

In [ ]:
log_likelihood({'a', 'b', 'c'}, {'d', 'e', 'f'}, 'abc', 'df')

-inf

### Subquestion h) - Decoding

In [ ]:
def to_semiring(fst, semiring):
    """ Lifts an WFST into a different semiring
    """
    oone, one = fst.R.one, semiring.one
    nfst = FST(semiring)

    # initial weights
    for q, w in fst.I:
        if w == oone:
            nfst.set_I(q, one)
        else:
            nfst.set_I(q, semiring(w.score))

    # final weights
    for q, w in fst.F:
        if w == oone:
            nfst.set_F(q, one)
        else:
            nfst.set_F(q, semiring(w.score))

    # arcs
    for p in fst.Q:
        for (a, b), q, w in fst.arcs(p):
            if w == oone:
                nfst.add_arc(p, a, b, q, one)
            else:
                nfst.add_arc(p, a, b, q, semiring(w.score))

    return nfst

In [ ]:
class Arctic(Semiring):
    def __init__(self, score):
        self.score = score

    def star(self):
        return self.one

    def __float__(self):
        return float(self.score)

    def __add__(self, other):
        return Arctic(max(self.score, other.score))

    def __mul__(self, other):
        if other is self.one:
            return self
        if self is self.one:
            return other
        if other is self.zero:
            return self.zero
        if self is self.zero:
            return self.zero
        return Arctic(self.score + other.score)

    def __invert__(self):
        return Arctic(-self.score)

    def __truediv__(self, other):
        return Arctic(self.score - other.score)

    def __lt__(self, other):
        return self.score < other.score

    def __repr__(self):
        return f"Arctic({self.score})"

    def __str__(self):
        return str(self.score)


Arctic.zero = Arctic(-float("inf"))
Arctic.one = Arctic(0.0)
Arctic.idempotent = True
Arctic.superior = True
Arctic.cancellative = True

In [ ]:
def decode(in_str, edit_distance_fst):
    """ Gets as input a source string and an
    edit-distance transducer and returns the weight of the most
    probable target string
    """
    # TODO: implement
    fst = string_fst(in_str)
    comp_fst = compose(fst, edit_distance_fst)

    lifted_fst = to_semiring(comp_fst, Arctic)
    weight = normalizer(lifted_fst).score

    return weight

In [ ]:
edit_distance = edit_distance_fst({'a', 'b', 'c'}, {'d', 'e'}, Log)
decode('abc', edit_distance)

-327.5889001988654